# Лекция 10. Последние штрихи 2

* Управление атрибутами
* match
* многозадачность

# Управление атрибутами

> `getattr(obj, attrname)` - функция, которая позволяет получить атрибут объекта по его имени

> `setattr(obj, attrname, value)` - функция, которая устанавливает значение атрибута объекта по его имени

In [139]:
class Test: pass

a = Test()

setattr(a, "X", 13)
# a.X = 13

# Оба вызова эквивалентны
print(a.X)
print(getattr(a, "X"))

13
13


При создании объектов можно перегрузить специальные методы, которые отвечают за доступ к необъявленным атрибутам класса

> `__getattr__(self, attrname)` - перехват доступа к необъявленым атрибутам для получения данных

> `__getattribute__(self, attrname)` - перехват доступа к любым атрибутам для получения данных

> `__setattr__(self, attrname, value)` - установка значения для атрибута

Работа с данным методами позволяет управлять доступ к различного рода атрибутом или даже создавать видимость их существования

In [51]:
class Test:
    def __getattr__(self, attrname):
        print("__getattr__")

        if attrname == "Exception":
            raise AttributeError
        else:
            return 20
    
    def __setattr__(self, attrname, value):
        print("set [%s] - %s" % (attrname, value))
        
a = Test()

In [52]:
a.Age

__getattr__


20

In [53]:
a.Exception

__getattr__


AttributeError: 

In [54]:
a.Weight = 1
a.Age = 1

set [Weight] - 1
set [Age] - 1


In [55]:
a.Weight

__getattr__


20

In [12]:
# НО! Здесь вызов данных методов не происходит

a.__dict__["Var"] = "Hi"
print(a.Var)

# setattr продолжает ничего не делать
a.Var = 123

Hi
set [Var] - 123


In [13]:
# а вот getattr перестал работать
a.Var

'Hi'

__Важно!__ При реализации метода `__setattr__` важно не использовать присваивание атрибутам, так как это приведет к зацикливанию кода

In [34]:
class Test:
    def __setattr__(self, attrname, value):
        self.__dict__[attrname] = value
        # а вот это бы привело к зацикливанию
        # self.value = value

a = Test()
a.Attr = 5
a.Attr

5

## Свойства

Python предоставляет удобный способ контроллируемого доступа к атрибутам объекта

> `property(getter, setter, deleter, docstring)` - данная функция позволяет создать атрибут-свойство. В качестве отсутствия обработчика на соответствующее действие можно передать None

In [56]:
class Test:        
    def get_age(self):
        return self._age
    
    def set_age(self, value):
        if value < 0:
            raise ValueError
        self._age = value
        
    def del_age(self):
        del self._age
        
    age = property(get_age, set_age, del_age, "It is age")
    
a = Test()

In [57]:
a.age

AttributeError: 'Test' object has no attribute '_age'

In [58]:
a.age = 20
a.age, a._age

(20, 20)

In [59]:
del a.age
a._age

AttributeError: 'Test' object has no attribute '_age'

Свойства позволяют контроллировать доступ к атрибутам объекта      

In [60]:
class Test:  
    def __init__(self):
        self._age = 20
        
    def get_age(self):
        return self._age     
    
    age = property(get_age, None, None, None)
    
a = Test()
print(a.age)
a.age = 25

20


AttributeError: can't set attribute 'age'

Также очень удобно использовать свойства через декораторы

In [191]:
class Test:
    @property
    def age(self):
        """Doc string"""
        return self._age

    @age.setter
    def age(self, value):
        self._age = value
        
    @age.deleter
    def age(self):
        del self._age

a = Test()
a.age = 15
print(a.age, a._age)

15 15


Чуть более реалистичный пример

In [37]:
class Distance:
    def __init__(self, distance):
        self._distance = distance
        
    @property
    def km(self): return self._distance / 1000

    @km.setter
    def km(self, value): self._distance = value * 1000
    
    @property
    def mm(self): return self._distance * 1000
    
    @property
    def m(self): return self._distance
    
d = Distance(1)

print(d.km, d.m, d.mm)

d.km = 5
d.m

0.001 1 1000


5000

# Примеры встроенных декораторов

> `staticmethod` - объявляет статический метод класса (метод, который не зависит от конкретного экземпляра класса). По сути обычная функция, которая не использует никаких данных из класса, в котором она объявлена. Такой метод можно вызывать как у самого класса, так и у его экземпляров.

In [39]:
class Test:
    @staticmethod
    def is_even(n):
        return n%2 == 0
    
print(Test.is_even(15))
a = Test()
print(a.is_even(15))

False
False


> `classmethod` - создает метод, который относится к классу, а не к экземпляру. Часто может быть использован, как перегрузка конструктора.

In [3]:
class Test:
    def __init__(self, num):
        self.var = num
        
    @classmethod
    def from_string(cls, string):
        # обратите внимание, cls - это ссылка на класс, а не на экземпляр
        return cls(int(string))
    
a = Test(13)
print(a.var)

a = Test.from_string("21")
print(a.var)

13
21


Менеджер контекста.

In [63]:
from contextlib import contextmanager

@contextmanager
def func():
    r = "context"
    f = open("tmp.log", "w")
    try:
        yield r
    finally:
        f.close()
        print("Close context")

with func() as e:
    print(e)

context
Close context


# `match`

Раньше для выбора из нескольких вариантов, например связанные с перечислениями, приходилось делать это следующей конструкцией

In [7]:
from enum import Enum

class Level:
    LOW=0
    MEDIUM=1
    HIGH=2

v = Level.MEDIUM

if v == Level.LOW:
    print("LOW")
elif v == Level.MEDIUM:    
    print("MEDIUM")
elif v == Level.HIGH:
    print("HIGH")    
else:
    print("unknown")

MEDIUM


Это весьма утомительно, поэтому к нам на помочь с Python 3.10 пришел на помощь
```Python
match expr:
    case pattern1:
        ...
    case pattern2:
        ...
    case _:
        ...
```

In [8]:
match v:
    case Level.LOW:
        print("LOW")
    case Level.MEDIUM:
        print("MEDIUM")
    case Level.HIGH:
        print("HIGH")
    case _:
        print("unknown")

MEDIUM


Шаблоны можно объединять вместе

In [9]:
match v:
    case Level.LOW | Level.MEDIUM:
        print("OK")
    case Level.HIGH:
        print("NOT OK")
    case _:
        print("unknown")

OK


## Кортежи и списки в качестве шаблона

Увы, это не единственное, что может делать `match`. В качестве шаблона для сравнения могут стоять кортежи, списки или словари (проверяется наличие ключей и их значений)

In [34]:

def Test(v):
    match v:
        case (1, 2):
            print("0: 1 2")
        case ("val1", "val2"):
            print("1: ", v)
        case ("val1", value):
            print("2: val1 ", value)
        case (val1, "val2"):
            print("3:", val1, "val2")
        case (val1, val2):
            print("3:", val1, val2)
        case _:
            print("_:", v)


Test((1, 2))
Test(("val1", "val2"))
Test(("val1", "---"))
Test(("---", "val2"))
Test(("---", "---"))
Test(("---", "---", 2))

0: 1 2
1:  ('val1', 'val2')
2: val1  ---
3: --- val2
3: --- ---
_: ('---', '---', 2)


Все также можно объединять их вместе или игнорировать некоторые значения

In [22]:

def Test(v):
    match v:
        case ("val1", "val2") | ("val11", "val22"):
            print("1: ", "val1, val2 or val11, val22")
        case ("val1" | "val11", val2):
            print("2: val1 or val11", val2)
        case ("val3", _):
            print("4: val3 and any")
        case (val1, "val2"):
            print("3:", val1, "val2")
        case (val1, val2):
            print("3:", val1, val2)
        case (val1, val2, val3):
            print("5:", val1, val2, val3)
        case _:
            print("_:", v)


Test(("val1", "val2"))
Test(("val11", "val22"))
Test(("val1", "---"))
Test(("val11", "---"))
Test(("---", "val2"))
Test(("---", "---"))
Test(("---", "---", 2))
Test(("---", "---", 2, 3))
Test(("val3", 2))

1:  val1, val2 or val11, val22
1:  val1, val2 or val11, val22
2: val1 or val11 ---
2: val1 or val11 ---
3: --- val2
3: --- ---
5: --- --- 2
_: ('---', '---', 2, 3)
4: val3 and any


Можно даже упаковывать части сравниваемых кортежей (для словарей `**`)


In [30]:

def Test(v):
    match v:        
        case ("val1" | "val11", *vals):
            print("1: val1 or val11", vals)
        case ("val2" | "val2", *_):
            print("2: val2 or val22")
        case _:
            print("_:", v)

Test(("val1", 1, 2, 3, 4))
Test(("val2", 1, 2, 3, 4))
Test(("val4", 1, 2, 3, 4))

1: val1 or val11 [1, 2, 3, 4]
2: val2 or val22
_: ('val4', 1, 2, 3, 4)


## Классы в качестве шаблона

Чтобы совсем запутать, то можно сравнивать по атрибутам экземпляров класса (обратите внимания, внутри case происходит сравнение, а не вызов конструктора, там передаются имена атрибутов)

In [45]:
class Vector:
    def __init__(self, xval, yval):
        self.x = xval
        self.y = yval

def Test(v):
    match v:
        case Vector(x=1, y=2):
            print("1:")
        case Vector(x=2 | 3, y=yval):
            print("2:", yval)
        case Vector(x=1, y=yval):
            print("3:", yval)
        case Vector(y=13):
            print("4:",)
        case _:
            print("_:")

Test(Vector(1, 2))
Test(Vector(3, 7))
Test(Vector(1, 5))
Test(Vector(155, 13))

1:
2: 7
3: 5
4:


Если очень хочется использовать позиционные аргументы

In [46]:
class Vector:
    __match_args__ = ("x", "y")
    def __init__(self, xval, yval):
        self.x = xval
        self.y = yval

def Test(v):
    match v:
        case Vector(1, 2):
            print("1:")
        case Vector(2 | 3, yval):
            print("2:", yval)
        case Vector(1, yval):
            print("3:", yval)
        case Vector(_, 13):
            print("4:",)
        case _:
            print("_:")

Test(Vector(1, 2))
Test(Vector(3, 7))
Test(Vector(1, 5))
Test(Vector(155, 13))

1:
2: 7
3: 5
4:


## Условия в шаблонах

Также можно добавлять условия

In [48]:
class Vector:
    def __init__(self, xval, yval):
        self.x = xval
        self.y = yval

def Test(v):
    match v:
        case Vector(x=1, y=yval) if yval < 10:
            print("1:", yval)
        case Vector(x=1, y=yval) if yval >= 10:
            print("2:", yval)
        case _:
            print("_:")

Test(Vector(1, 2))
Test(Vector(1, 15))
Test(Vector(2, 5))

1: 2
2: 15
_:


## `as`

Можно совпадающее с шаблоном значение поместить в переменную

In [49]:
match v:
    case Level.LOW | Level.MEDIUM as level:
        print("OK: ", level)
    case Level.HIGH:
        print("NOT OK")
    case _:
        print("unknown")

OK:  1


# Домашняя работа

# Задача 1

Написать класс, который реализует обобщенное хранение целого числа в себе. Добавить свойства, которые позволяют получить хранимое в нем число в десятичной записи, двоичной записи и римской записи. Добавить методы класса, которые позволяют получить экземпляры этого класса из разных форматов записи чисел (from_dec, from_roman и прочее).

In [35]:
class Number:
    ...


n = Number(401)
n = n + Number(100)

# примеры свойств
print(n.dec)
print(n.bin)
print(n.hex)
print(n.roman)

501
0b111110101
0x1f5
DI
